In [ ]:
# Create the interface
from abc import ABC, abstractmethod

class DigitClassificationInterface(ABC):

    @abstractmethod
    def predict(self, image):
        pass

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Example model CNN
mnist = fetch_openml('mnist_784')
X, y = mnist.data, mnist.target
y = y.astype(np.int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_cnn = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255
X_test_cnn = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255
y_train_cnn = to_categorical(y_train, 10)
y_test_cnn = to_categorical(y_test, 10)

# Creat model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_cnn, y_train_cnn, validation_data=(X_test_cnn, y_test_cnn), epochs=10, batch_size=200)
score = model.evaluate(X_test_cnn, y_test_cnn, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
model.save('cnn_model.h5')


class CNNModel(DigitClassificationInterface):
    def __init__(self):
        # Assuming that the model is saved in 'cnn_model.h5'
        self.model = load_model('cnn_model.h5')

    def predict(self, image):
        image = image.reshape((1, 28, 28, 1))
        prediction = self.model.predict(image)
        return np.argmax(prediction)

# Example model RandomForest
mnist = fetch_openml('mnist_784')
X, y = mnist.data, mnist.target
y = y.astype(np.int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creat model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train)
y_pred = rf_model.predict(X_test_rf)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy}')
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

class RFModel(DigitClassificationInterface):
    def __init__(self):
        # Assuming that the model is saved in 'rf_model.pkl'
        import pickle
        with pickle.open('rf_model.pkl', 'rb') as f:
            self.model = pickle.load(f)

    def predict(self, image):
        image = image.reshape((1, -1))  
        return self.model.predict(image)[0]

class RandomModel(DigitClassificationInterface):
    def predict(self, image):
        return np.random.randint(0, 10)